In [1]:
## Vacation Days  = 1/10/12 - 1/15/12

In [2]:
# import dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, MetaData, Table, func, inspect, extract

import pandas as pd
import matplotlib
from matplotlib import style
style.use('seaborn')
import matplotlib.pyplot as plt
import numpy as np

Base = automap_base()


In [3]:
# create engine
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect the tables
Base.prepare(engine, reflect=True)


In [4]:
# verify classes are alive
Base.classes.keys()                                                          # inspects database


['measurements', 'stations']

In [5]:
# mapped classes are now created with names by default
# matching that of the table name.
Measurement = Base.classes.measurements
Station = Base.classes.stations


In [6]:
inspector = inspect(engine)
inspector.get_table_names()

['measurements', 'stations']

In [7]:
# view columns of measurements
columns = inspector.get_columns('measurements')
for c in columns:
    print(c)

{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}
{'name': 'station', 'type': VARCHAR(length=255), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'date', 'type': VARCHAR(length=255), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'prcp', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'tobs', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}


In [8]:
# view first 10 rows of Measurement
engine.execute('SELECT * FROM measurements LIMIT 10').fetchall()


[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-06', 0.0, 73),
 (6, 'USC00519397', '2010-01-07', 0.06, 70),
 (7, 'USC00519397', '2010-01-08', 0.0, 64),
 (8, 'USC00519397', '2010-01-09', 0.0, 68),
 (9, 'USC00519397', '2010-01-10', 0.0, 73),
 (10, 'USC00519397', '2010-01-11', 0.01, 64)]

In [9]:
# Start a session to query the database
session = Session(engine)

### Precipitation Analysis

In [10]:
# retrieve last 12 months of precipitation data
precipitation_data = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date>="2011-01-09").filter(Measurement.date<="2012-01-09").all()


In [11]:
# load query into df
precipitation_data_df = pd.DataFrame(precipitation_data)
precipitation_data_df.head()

,date,prcp
0,2011-01-09,0.00
1,2011-01-10,0.00
2,2011-01-11,0.09
3,2011-01-13,0.00
4,2011-01-14,0.03


In [12]:
# plot temperature


In [13]:
# describe data
precipitation_data_df.describe()

,prcp
count,2732.000000
mean,0.152873
std,0.474107
min,0.000000
25%,0.000000
50%,0.010000
75%,0.092500
max,8.810000


### Station Analysis

In [14]:
# query number of stations
station_count = session.query(Station.id).count()
station_count

9

In [15]:
# query most active stations, desc
station_activity = session.query(Measurement.station, Station.name,\
                                 func.count(Measurement.tobs)).filter(Measurement.station == Station.station).\
                                group_by(Measurement.station).order_by(func.count(Measurement.tobs).desc()).all()
station_activity


[('USC00519281', 'WAIHEE 837.5, HI US', 2772),
 ('USC00519397', 'WAIKIKI 717.2, HI US', 2724),
 ('USC00513117', 'KANEOHE 838.1, HI US', 2709),
 ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 2669),
 ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 2612),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 2202),
 ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 1979),
 ('USC00517948', 'PEARL CITY, HI US', 1372),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 511)]

In [16]:
# plot station data


### Temperature Analysis

In [17]:
def calc_temps(start_date, end_date):
    
    """ accepts a start date and end date in the format `%Y-%m-%d` and returns the 
    minimum, average, and maximum temperatures for that range of dates."""
    # create empty list for temp data
    temp_stats = []
    # query rows from selected timeframe
    query = session.query(Measurement.tobs).filter(Measurement.date>=start_date, Measurement.date <= end_date).all()
    
    temps = [temp[0] for temp in query]
    temp_stats.append(np.mean(temps))
    temp_stats.append(min(temps))
    temp_stats.append(max(temps))
    
    return temp_stats

In [18]:
## Vacation Days  = 1/10/12 - 1/15/12
calc_temps('2011-01-10','2011-01-15')



[68.043478260869563, 57, 76]

In [19]:
# plot temp analysis
